In [7]:
import fastmri
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pathlib
import pytorch_lightning as pl

import os
from torch.utils.data import DataLoader
from fastmri.data import transforms as T
from fastmri.data.subsample import create_mask_for_mask_type, RandomMaskFunc
from fastmri.data.mri_data import fetch_dir
from fastmri.pl_modules import FastMriDataModule
from src.unet.unet_module import UnetModule
from argparse import ArgumentParser



In [8]:
path_config = pathlib.Path("fastmri_dirs.yaml")

configs = dict(
    challenge="singlecoil",
    num_gpus=1,
    backend="mps",
    batch_size=1,
    data_path=fetch_dir("knee_path", path_config),
    default_root_dir=fetch_dir("log_path", path_config) / "unet" / "unet_demo",
    mode="train",  # "train" or "test"
    mask_type="random",  # "random" or "equispaced_fraction"
    center_fractions=[0.08],  # number of center lines to use in the mask
    accelerations=[4],  # acceleration rates to use for the mask
    # model parameters
    in_chans=1,
    out_chans=1,
    chans=32,
    num_pool_layers=4,
    drop_prob=0.0,
    lr=0.001,
    lr_step_size=40,
    lr_gamma=0.1,
    weight_decay=0.0,
    max_epochs=50
)

pl.seed_everything(42)

# mask for transforming the input data
mask = create_mask_for_mask_type(
    configs['mask_type'], configs['center_fractions'], configs['accelerations']
)

# random masks for train, fixed masks for val
train_transform = T.UnetDataTransform(configs['challenge'],
                                      mask_func=mask,
                                      use_seed=False)
val_transform = T.UnetDataTransform(configs['challenge'], mask_func=mask)
test_transform = T.UnetDataTransform(configs['challenge'])

# create a data module
data_module = FastMriDataModule(
    data_path=configs['data_path'],
    challenge=configs['challenge'],
    train_transform=train_transform,
    val_transform=val_transform,
    test_transform=test_transform,
    test_path=None,
    batch_size=configs['batch_size'],
    num_workers=4,
)

# create a model
model = UnetModule(
    in_chans=configs['in_chans'],
    out_chans=configs['out_chans'],
    chans=configs['chans'],
    num_pool_layers=configs['num_pool_layers'],
    drop_prob=configs['drop_prob'],
    lr=configs['lr'],
    lr_step_size=configs['lr_step_size'],
    lr_gamma=configs['lr_gamma'],
    weight_decay=configs['weight_decay'],
)

callbacks = [
    pl.callbacks.ModelCheckpoint(
        dirpath=configs['default_root_dir'],
        monitor="val_loss",
        mode="min",
        save_top_k=1,
        save_last=True,
        verbose=True,
    ),
    pl.callbacks.LearningRateMonitor(logging_interval="epoch"),
]

# create a trainer
trainer = pl.Trainer(
    devices=configs['num_gpus'],
    max_epochs=configs['max_epochs'],
    default_root_dir=configs['default_root_dir'],
    accelerator=configs['backend'],
    callbacks=callbacks,
)



#if args.resume_from_checkpoint is None:
#    ckpt_list = sorted(checkpoint_dir.glob("*.ckpt"), key=os.path.getmtime)
  #  if ckpt_list:
 #       args.resume_from_checkpoint = str(ckpt_list[-1])

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model, datamodule=data_module)

Missing logger folder: logs/unet/unet_demo/lightning_logs


KeyError: "Unable to open object (object 'ismrmrd_header' doesn't exist)"